In [1]:
from math import cos, sin, tan, acos, asin, atan, atan2, degrees, radians
from numpy import reciprocal, pi, sqrt, angle, exp
from scipy import *
from scipy.constants import k, e
import numpy as np
import scipy as sp
import time, sys

#hextime=float.hex(time.time())  # reserved for a version revision 


def cosd(x):
  return cos(radians(x))

def sind(x):
  return sin(radians(x))

def tand(x):
  return tan(radians(x))

def acosd(x):
  return degrees(acos(x))

def asind(x):
  return degrees(asin(x))

def atand(x):
  return degrees(atan(x))

def atan2d(y, x):
  return degrees(atan2(y, x))

## Now define conversion functions:

def P2R(radii, angles):
    return radii * exp(1j*angles*(pi/180))

def R2P(x):
    return abs(x), angle(x, deg=True)

print("Author         : S Monroe")
print("Version        :",sys.version[0:66])
print("Numeric    lib :",np.version.version)
print("Scientific lib :",sp.version.version)
print("Encoding       :",sys.getdefaultencoding())
print("Timestamp      :",time.time())

Author         : S Monroe
Version        : 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Numeric    lib : 1.18.1
Scientific lib : 1.5.0
Encoding       : utf-8
Timestamp      : 1600219481.0079603


In [2]:
VCC=input("Enter VCC:")
VCC=float(VCC)
R1=input("Enter R1:")
R1=float(R1)
R2=input("Enter R2:")
R2=float(R2)
RC=input("Enter RC:")
RC=float(RC)
RE=input("Enter RE:")
RE=float(RE)
Temp=input("Enter ambient temperature (degrees C:)")
Temp=float(Temp)
β=input("Enter β:")
β=float(β)
RL=input("Enter load:")
RL=float(RL)

## Calculate β*RE ratio: drives how IC is calculated:

if β*RE >= 10.0*R2:
    VTH=(VCC)*(R2/(R1+R2))
    RTH=np.reciprocal(np.reciprocal(R1)+np.reciprocal(R2))
    VE=VTH-0.7
    IC=VE/RE
else:
    VTH=(VCC)*(R2/(R1+R2))
    RTH=np.reciprocal(np.reciprocal(R1)+np.reciprocal(R2))
    IC=(VTH-0.7)/((RTH/β)+RE)

Enter VCC:22
Enter R1:128000
Enter R2:10000
Enter RC:5600
Enter RE:470
Enter ambient temperature (degrees C:)27
Enter β:120
Enter load:8200


In [3]:
# calculate r'e ...
T=Temp+273.15
VT=((k*T)/e)
VBE=700.0E-3

In [4]:
V2=VTH
V1=VCC-V2
IB=(VTH-VBE)/(RTH+(β+1)*RE)
#IB=IC/β      # a more approximate answer
#VE=V2-0.7    # a more approximate answer
#IE=VE/RE     # a more approximate answer
IE=IB*(β+1)   # accounts for base leakage
VE=IE*RE
VCE=VCC-(IC*RC)+(IE*RE) #accommodates difference between IC and IE due to IB leakage.
VC=VE+VCE
VRC=VCC-VC
r_prime_e=VT/IE
r_c=np.reciprocal(np.reciprocal(RC)+np.reciprocal(RL))
A_v=r_c/r_prime_e
A_v_db=20.0*np.log10(A_v)
z_in_base=β*r_prime_e
z_in=np.reciprocal(np.reciprocal(R1)+np.reciprocal(R2)+np.reciprocal(z_in_base))

In [5]:
β_calc=IC/IB
print("β(dc) Calculated value:",β_calc)
β_err=abs((β-β_calc)/β)*100.0
β_err=β_err*10000.0
print("Absolute Uncertainty in β(dc):",'{0:+06.2f}'.format(float(β_err)),"ppm")

β(dc) Calculated value: 120.85876952952599
Absolute Uncertainty in β(dc): +7156.41 ppm


In [6]:
gm=IC/VT
r_π=VT/IB
β_alt=gm*r_π
print(gm)
print(r_π)
print(β_alt)

0.06316894232728636
1913.262515989919
120.85876952952599


In [7]:
print(β*RE)
print(10.0*R2)
print("UCL for VCE:",VCC*0.75,"V")
print("LCL for VCE:",VCC*0.25,"V")
print("VT @",Temp,"degrees C :",VT,"V")
print("V_R1:",V1,"V")
print("V_R2:",V2,"V")
print("V_RC:",VRC,"V")
print(" V_C:",VC,"V")
print(" V_B:",V2,"V")
print(" V_E:",VE,"V")
print(" VCE:",VCE,"V")
print("  IC:",IC,"A")
print("  IE:",IE,"A")
print("  IB:",IB,"A")
print(" r'e:",r_prime_e,"\u03A9")
print("  rc:",r_c,"\u03A9")
print("z_in:",z_in,"\u03A9")
print(" A_v:",A_v)
print(" A_v:",A_v_db,"dB")

56400.0
100000.0
UCL for VCE: 16.5 V
LCL for VCE: 5.5 V
VT @ 27.0 degrees C : 0.025864925786328753 V
V_R1: 20.405797101449274 V
V_R2: 1.5942028985507246 V
V_RC: 7.611992920257277 V
 V_C: 14.388007079742723 V
 V_B: 1.5942028985507246 V
 V_E: 0.7688115547005608 V
 VCE: 13.619195525042162 V
  IC: 0.0016338600052961426 A
  IE: 0.0016357692653203421 A
  IB: 1.3518754258845802e-05 A
 r'e: 15.812086909007595 Ω
  rc: 3327.5362318840585 Ω
z_in: 1575.2112389996928 Ω
 A_v: 210.44257162465235
 A_v: 46.462672005400364 dB


In [8]:
print("KVL Check:")
VKVL=VE+VCE+VRC
print("Σ V:",VKVL,"V")
VKVL==VCC

KVL Check:
Σ V: 22.0 V


True

In [9]:
VA=100 # typical values are 20V-180V --> use median (50%-ile) value.  NB! This value is process/part dependent.
ro=(VA+VCE)/IC
print("ro:",ro,"\u03A9")

ro: 69540.34933026486 Ω


Early Voltage for some modeled parts:

<code>
2N2222A: 104V<br>
2N5172: 37.37V<br>
2N4124: 74.03V<br>
2N3904: 30V<br>

An example of an outlier:
<pre>
.SUBCKT dtc144erlra/ON 1 2 3 
* MODEL FORMAT: PSpice
* BRT macro model
* External node 
* Node 1 -> BRT_C
* Node 2 -> BRT_B
* Node 3 -> BRT_E
QOUT 1 4 3 qmodel
R1   2 4 47000
R2   4 3 47000
.MODEL qmodel npn
+IS=1.00001e-17 BF=349.534 NF=0.839361 VAF=804.744
+IKF=0.306026 ISE=1.0012e-16 NE=1.15668 BR=0.100001
+NR=0.950889 VAR=1000 IKR=0.000126051 ISC=1.00015e-16
+NC=4 RB=0.1 IRB=0.2 RBM=1
+RE=3.05485 RC=0.001 XTB=0.939521 XTI=2.41603 EG=1.206
+CJE=1.01139e-13 VJE=0.685626 MJE=0.403552 TF=1e-09
+XTF=1 VTF=10 ITF=0.01 CJC=1.04743e-12
+VJC=10.1219 MJC=0.856202 XCJC=0.9 FC=0.5
.ENDS.SUBCKT dtc144erlra/ON 1 2 3</pre>